In [ ]:
# Required for Google Colab!

!pip install rasterio
!pip install xarray
!pip install zarr

from google.colab import drive
drive.mount('/content/drive')

import os

# Root of project
os.chdir('/content/drive/MyDrive/Final Submission Files')

# Improvements folder
os.chdir('Models/Improvements')

In [ ]:
import xarray as xr
import numpy as np
import datetime
import pandas as pd
import glob
import os
import pickle
import copy 

from keras.regularizers import L1L2

from model import *

In [ ]:
# Read spatial data and extents

extents = xr.open_dataarray('../../Data/Improvements/extents.nc')
spatial_ds = xr.open_zarr('../../Data/Improvements/ds_norm_coarsened_4x.zarr')

In [ ]:
# Define a list of parameters to test
all_model_params = [
    dict(
        num_convlstm=3,
        convlstm_filters=(64, 32, 32),
        convlstm_kernels=((5, 5), (3, 3), (1, 1)),
        convlstm_dropout=0.1,
        convlstm_rec_dropout=0.1,
        convlstm_kernel_reg=L1L2(0.001, 0.001),
        batchnorm=True,
        num_conv2d=2,
        conv2d_filters=(32, 1),
        conv2d_kernels=((3, 3), (3, 3)),
        max_pooling=True,
        num_lstm=3,
        lstm_units=(64, 32, 1),
        lstm_dropout=0.1,
        lstm_kernel_reg=L1L2(0.001, 0.001),
        learning_rate=0.0001,
        GAP=False,
    ),
    dict(
        num_convlstm=3,
        convlstm_filters=(128, 64, 64),
        convlstm_kernels=((5, 5), (3, 3), (1, 1)),
        convlstm_dropout=0.1,
        convlstm_rec_dropout=0.1,
        convlstm_kernel_reg=L1L2(0.001, 0.001),
        batchnorm=True,
        num_conv2d=2,
        conv2d_filters=(64, 64),
        conv2d_kernels=((3, 3), (3, 3)),
        max_pooling=True,
        num_lstm=3,
        lstm_units=(128, 64, 1),
        lstm_dropout=0.1,
        lstm_kernel_reg=L1L2(0.001, 0.001),
        learning_rate=0.0001,
        GAP=False,
    )
]

In [ ]:
# Train the models using 3, 5 and 7 input timesteps

# Directory in which to save results
save_dir = '../../Results/Improvements/test/'

for num_timesteps in [3, 5, 7]:

    # Create model object and pre-process data (only once for each num_timesteps)
    model = Spatial()
    data = model.process_data(
        spatial_ds,
        extents,
        num_timesteps=num_timesteps,
        deseasonalize_type=1,
        test_size=0.2
        )
    
    for model_num, model_params in enumerate(all_model_params):
        try:
            # Model name
            model_name = f'{model_num}_{num_timesteps}T'

            # Create model
            model.create_model(**model_params)

            # Train
            model.train(data,
                        model_name=model_name,
                        save_dir=save_dir,
                        epochs=100,
                        verbose=1)

            # Load best model
            model.model.load_weights(model.checkpoint_filepath)

            # Get predictions & add to data dictionary
            model_data = copy.deepcopy(data)
            model_data['y_pred_test'] = model.model.predict([data['X_grid_test'], data['X_extent_test']])
            model_data['y_pred_train'] = model.model.predict([data['X_grid_train'], data['X_extent_train']])

            # Delete gridded input data from data dictionary (too big)
            del model_data['X_grid_test']
            del model_data['X_grid_train']

            # Add model_params & results as attribute to object before saving
            model.model_params = model_params
            model.data = model_data

            # Save as pickle & save model
            model.save(model_name, save_dir)
            print(f'Saved {model_name}!')
        except Exception as e:
            print('Could not create model!')
            print(e)

        break
    break

Epoch 1/100
39/39 [==============================] - 24s 309ms/step - loss: 6.6494 - val_loss: 6.7721 - lr: 1.0000e-04
Epoch 2/100
39/39 [==============================] - 10s 247ms/step - loss: 6.3443 - val_loss: 6.7030 - lr: 1.0000e-04
Epoch 3/100
39/39 [==============================] - 10s 246ms/step - loss: 6.2520 - val_loss: 6.6317 - lr: 1.0000e-04
Epoch 4/100
39/39 [==============================] - 10s 246ms/step - loss: 6.1463 - val_loss: 6.5621 - lr: 1.0000e-04
Epoch 5/100
39/39 [==============================] - 10s 247ms/step - loss: 6.0007 - val_loss: 6.4921 - lr: 1.0000e-04
Epoch 6/100
39/39 [==============================] - 10s 247ms/step - loss: 5.9041 - val_loss: 6.4118 - lr: 1.0000e-04
Epoch 7/100
39/39 [==============================] - 10s 247ms/step - loss: 5.7880 - val_loss: 6.3104 - lr: 1.0000e-04
Epoch 8/100
39/39 [==============================] - 10s 244ms/step - loss: 5.6179 - val_loss: 6.3178 - lr: 1.0000e-04
Epoch 9/100
39/39 [=============================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Project/Results/test/0_3T_model.p/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Project/Results/test/0_3T_model.p/assets


Saved 0_3T!
